In [ ]:
# ADJUST THIS FOR YOUR MACHINE
output_directory_path = "results"  # Directory to save results in

In [ ]:
import tqdm
import pathlib
import yaml
import itertools
import numpy as np

In [ ]:
# Each configuration file consists of constant and variable paramaters.
# Constant parameters are equal across all simulations, variable parameters are not.
# We are interested in the effect of variable parameters on the evolution of life history.
# Variable parameters are:
#   - `MAX_POPULATION_SIZE`
#   - `REPRODUCTION_MODE`
#   - `PHENOMAP_SPECS`. 
#       This is used to encode two kinds of mutational effects: 
#       detrimental monotropic (theory of mutation accumulation) or 
#       antagonistic pleiotropic (theory of antagonistic pleiotropy) effects.

In [ ]:
# MACHINE-SPECIFIC PARAMETERS

path_dir = pathlib.Path(output_directory_path)
path_dir.mkdir(exist_ok=True)  # Create the directory if it does not exist

In [ ]:
# CONSTANT PARAMETERS

params_const = {
    # General
    "STAGES_PER_SIMULATION_": 2000000,
    "RANDOM_SEED_": 42,
    #
    # Life history
    "MATURATION_AGE": 10,
    "MAX_LIFESPAN": 50,
    #
    # Genome structure
    "BITS_PER_LOCUS": 1,
    "G_neut_evolvable": True,
    "G_neut_lo": 0,
    "G_neut_hi": 1,
    "G_neut_initial": 0,
    "G_neut_interpreter": "single_bit",
    "G_repr_interpreter": "const1",
    "G_surv_interpreter": "const1",
    "G_repr_hi": 0.3,  # reproduction rate
    "G_neut_agespecific": 2500,  # number of mutating loci
    "G_muta_initial": 0.00001,  # mutation rate
    "MUTATION_RATIO": 100,
    #
    # Recording
    "PICKLE_RATE_": 100000,
    "POPGENSTATS_RATE_": 0,
    "SNAPSHOT_RATE_": 100000,
    "VISOR_RATE_": 10000,
}

In [ ]:
# VARIABLE PARAMETERS
REPRODUCTION_MODEs = ("sexual", "asexual_diploid")
aging_theories = ("MA", "AP")

In [ ]:
# EXPERIMENT-SPECIFIC PARAMETERS

# When generating data for figures 2 and 3
n_specs_studied = 30
MAX_POPULATION_SIZEs = (300, 3000, 30000)

# When generating data for figure 4
n_specs_studied = 3
MAX_POPULATION_SIZEs = (
    100,
    200,
    300,
    400,
    500,
    600,
    700,
    800,
    900,
    1000,
    1200,
    1500,
    2000,
    2500,
    3000,
    30000,
)

In [ ]:
# Generate phenomaps
def get_phenomap_specs(RANDOM_SEED_):
    """Generate MA and AP PHENOMAP_SPECS.
    MA specs are equivalent to AP specs but without the beneficial effects."""

    # Random number generator
    rng = np.random.default_rng(RANDOM_SEED_)

    # Number of mutable loci
    N = params_const["G_neut_agespecific"]

    # Set initial survival rate to 0.95
    bckg_specs = []
    for i in range(1, params_const["MAX_LIFESPAN"] + 1):
        bckg_specs.append(["surv", i, "surv", str(i), str(0.95)])

    # Generate N magnitudes of effects
    magns = rng.exponential(1, size=N) * 0.0014

    # Generate N ages at which the effects act
    # Note that it is possible that tn=tn=1
    timings_n = rng.integers(1, params_const["MAX_LIFESPAN"], size=N, endpoint=True)
    timings_p = rng.integers(1, timings_n, size=N, endpoint=True)

    # Assemble magnitudes and timings into specifications
    ma_specs = []
    ap_specs = []
    pos_specs = []

    neut_i = 1
    for m, tp, tn in zip(magns, timings_p, timings_n):
        ap_specs.append(["neut", neut_i, "surv", str(tp), str(m)])
        ap_specs.append(["neut", neut_i, "surv", str(tn), str(-m)])
        neut_i += 1

    neut_i = 1
    for m, tn in zip(magns, timings_n):
        ma_specs.append(["neut", neut_i, "surv", str(tn), str(-m)])
        neut_i += 1

    neut_i = 1
    # Use negative timings because it is uniformly distributed
    for m, tn in zip(magns, timings_n):
        pos_specs.append(["neut", neut_i, "surv", str(tn), str(m)])
        neut_i += 1

    return {
        "MA": ma_specs + bckg_specs,
        "AP": ap_specs + bckg_specs,
        "pos": pos_specs + bckg_specs,
    }


PHENOMAP_SPECS_list = [get_phenomap_specs(seed) for seed in range(n_specs_studied)]

In [ ]:
# Calculate total number of experiments
tqdm_total = (
    len(PHENOMAP_SPECS_list)
    * len(aging_theories)
    * len(REPRODUCTION_MODEs)
    * len(MAX_POPULATION_SIZEs)
)
print(f"Total {tqdm_total} experiments.")

In [ ]:
# Generate config files

params = itertools.product(
    range(len(PHENOMAP_SPECS_list)),
    aging_theories,
    REPRODUCTION_MODEs,
    MAX_POPULATION_SIZEs,
)

with tqdm.tqdm(total=tqdm_total) as pbar:

    # Iterate over combinations of variable input parameters
    for i, [
        phenomap_i,
        aging_theory,
        REPRODUCTION_MODE,
        MAX_POPULATION_SIZE,
    ] in enumerate(params):

        PHENOMAP_SPECS = PHENOMAP_SPECS_list[phenomap_i][aging_theory]

        # Construct final dictionary containing parameter keys and values
        params_final = params_const.copy()
        params_final["MAX_POPULATION_SIZE"] = MAX_POPULATION_SIZE
        params_final["REPRODUCTION_MODE"] = REPRODUCTION_MODE
        params_final["RECOMBINATION_RATE"] = 0.5 if REPRODUCTION_MODE == "sexual" else 0
        params_final["PHENOMAP_SPECS"] = PHENOMAP_SPECS

        # The ratio of 0->1 vs 1->0 is inverted for when simulation scenario is positive mutations
        if aging_theory == "pos":
            params_final["MUTATION_RATIO"] = 1 / params_const["MUTATION_RATIO"]

        # Write the yml to output directory
        yml_path = (
            path_dir
            / f"{aging_theory}-{MAX_POPULATION_SIZE}-{REPRODUCTION_MODE}-{i}.yml"
        )

        with open(yml_path, "w") as file_:
            yaml.dump(params_final, file_, sort_keys=False, default_flow_style=True)

        # Update the progress bar
        pbar.update(1)

In [ ]:
yml_paths = [
    path for path in path_dir.glob("*") if path.suffix == ".yml"
]  # Paths to all generated configuration files

# Simulations can be run by executing the following command for each configuration file:
# $ python3 -m aegis {yml_path}